In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import random
import os

In [2]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

In [3]:
def create_sequences(data, seq_length, feature_cols, target_col):
    xs, ys = [], []
    for _, group in data.groupby('driverId'):
        group = group.sort_values(['year', 'round']) 
        features = group[feature_cols].values
        targets = group[target_col].values
        
        if len(group) <= seq_length:
            continue
            
        for i in range(len(group) - seq_length):
            xs.append(features[i:(i + seq_length)])
            ys.append(targets[i + seq_length])
            
    return np.array(xs), np.array(ys)


In [4]:
class F1Dataset(Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32).view(-1, 1)
        
    def __len__(self): return len(self.x)
    def __getitem__(self, idx): return self.x[idx], self.y[idx]


In [5]:
class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super(Attention, self).__init__()
        self.attn = nn.Linear(hidden_dim, 1)

    def forward(self, lstm_output):
        attn_weights = torch.softmax(self.attn(lstm_output), dim=1)
        context = torch.sum(attn_weights * lstm_output, dim=1)
        return context, attn_weights


In [6]:
class F1SprintPredictor(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers=2):
        super(F1SprintPredictor, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=0.2)
        self.attention = Attention(hidden_dim)
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        context, _ = self.attention(lstm_out)
        return self.fc(context)


In [7]:
def train_and_evaluate():
    
    file_path = '../data_engineered/modern_train_ready.csv'
    if not os.path.exists(file_path):
        print("Error: No Engineered data found. Please run the preprocessing script first.")
        return

    
    model_dir = '../models'
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)

    df = pd.read_csv(file_path)
    
    
    FEATURE_COLS = ['grid', 'qual_pos', 'q_rel_gap', 'dob', 'constructorId', 'circuitId', 'pit_count']
    TARGET_COL = 'sprint_target'
    SEQ_LENGTH = 3
    
    
    X, y = create_sequences(df, SEQ_LENGTH, FEATURE_COLS, TARGET_COL)
    print(f"Number of sequences created: {X.shape[0]}")

    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    train_loader = DataLoader(F1Dataset(X_train, y_train), batch_size=16, shuffle=True)
    test_loader = DataLoader(F1Dataset(X_test, y_test), batch_size=1)

    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = F1SprintPredictor(input_dim=len(FEATURE_COLS), hidden_dim=64).to(device)
    
    criterion_mse = nn.MSELoss()
    criterion_mae = nn.L1Loss() 
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    
    EPOCHS = 100
    print(f"Training started (Device: {device})...")
    
    for epoch in range(EPOCHS):
        model.train()
        train_loss = 0
        for batch_x, batch_y in train_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion_mse(outputs, batch_y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            
        
        if (epoch + 1) % 20 == 0:
            model.eval()
            val_mse = 0
            val_mae = 0
            with torch.no_grad():
                for val_x, val_y in test_loader:
                    val_x, val_y = val_x.to(device), val_y.to(device)
                    preds = model(val_x)
                    val_mse += criterion_mse(preds, val_y).item()
                    val_mae += criterion_mae(preds, val_y).item()
            
            avg_val_mse = val_mse / len(test_loader)
            avg_val_mae = val_mae / len(test_loader)
            avg_val_rmse = np.sqrt(avg_val_mse)
            
            print(f"Epoch [{epoch+1:3d}/{EPOCHS}] Train Loss: {train_loss/len(train_loader):.4f}")
            print(f"          >> Val MSE: {avg_val_mse:.4f} | Val RMSE: {avg_val_rmse:.4f} | Val MAE: {avg_val_mae:.4f}")

    
    model.eval()
    all_preds = []
    all_actuals = []
    
    with torch.no_grad():
        for batch_x, batch_y in test_loader:
            batch_x = batch_x.to(device)
            preds = model(batch_x)
            all_preds.append(preds.item())
            all_actuals.append(batch_y.item())

    mae = mean_absolute_error(all_actuals, all_preds)
    mse = mean_squared_error(all_actuals, all_preds)
    rmse = np.sqrt(mse)

    print("\n" + "="*50)
    print(f"Final Model Evaluation Report (Random Seed: 42)")
    print(f"1. Mean Absolute Error (MAE)  : {mae:.2f} positions")
    print(f"2. Mean Squared Error (MSE)  : {mse:.4f}")
    print(f"3. Root Mean Squared Error (RMSE): {rmse:.2f} positions")
    print(f"\nInterpretation: On average, the model predicts results within ±{mae:.2f} positions.")
    print("="*50)

    
    save_path = os.path.join(model_dir, 'f1_sprint_predictor.pth')
    torch.save(model.state_dict(), save_path)
    print(f"Model saved successfully: {save_path}")

In [8]:
if __name__ == "__main__":
    train_and_evaluate()

Number of sequences created: 273
Training started (Device: cuda)...
Epoch [ 20/100] Train Loss: 20.4880
          >> Val MSE: 18.8689 | Val RMSE: 4.3438 | Val MAE: 3.5693
Epoch [ 40/100] Train Loss: 20.0393
          >> Val MSE: 17.8242 | Val RMSE: 4.2219 | Val MAE: 3.5579
Epoch [ 60/100] Train Loss: 18.6207
          >> Val MSE: 17.6000 | Val RMSE: 4.1952 | Val MAE: 3.5167
Epoch [ 80/100] Train Loss: 18.0867
          >> Val MSE: 17.6185 | Val RMSE: 4.1974 | Val MAE: 3.5250
Epoch [100/100] Train Loss: 19.0414
          >> Val MSE: 17.5707 | Val RMSE: 4.1917 | Val MAE: 3.4581

Final Model Evaluation Report (Random Seed: 42)
1. Mean Absolute Error (MAE)  : 3.46 positions
2. Mean Squared Error (MSE)  : 17.5707
3. Root Mean Squared Error (RMSE): 4.19 positions

Interpretation: On average, the model predicts results within ±3.46 positions.
Model saved successfully: ../models/f1_sprint_predictor.pth
